
http://jira.lge.com/issue/browse/HEDATAPLFM-999?attachmentSortBy=dateTime&attachmentOrder=asc

In [0]:
%sql
SELECT 
  date_ym, 
  log_create_time, 
  mac_addr,
  X_User_Number,
  get_json_object(normal_log, '$.old') as old_state,
  get_json_object(normal_log, '$.new') as new_state,
  CASE 
    WHEN (get_json_object(normal_log, '$.old') IN ('Init', 'Active Standby', 'Always Ready', 'Suspend') 
          AND get_json_object(normal_log, '$.new') IN ('Active'))
    THEN 'TV_ON'
    WHEN (get_json_object(normal_log, '$.old') IN ('Active', 'Screen Saver', 'Screen Off') 
          AND get_json_object(normal_log, '$.new') IN ('Active Standby', 'Always Ready', 'Suspend', 'Power Off', 'Reboot'))
    THEN 'TV_OFF'
    ELSE ''
  END AS tv_onoff_event
FROM kic_data_ods.tlamp.normal_log_webos25
WHERE 1=1
  AND date_ym = '2025-06'
  AND context_name = 'tvpowerd'
  AND message_id = 'NL_POWER_STATE'
  AND log_create_time >= to_date(concat(date_ym, '-01'))
ORDER BY mac_addr, X_User_Number, log_create_time


In [0]:
%sql
WITH ev AS (
  SELECT
    date_ym,
    CAST(log_create_time AS TIMESTAMP) AS ts,
    mac_addr,
    X_User_Number,
    CASE 
      WHEN get_json_object(normal_log, '$.old') IN ('Init','Active Standby','Always Ready','Suspend')
           AND get_json_object(normal_log, '$.new') IN ('Active') THEN 'TV_ON'
      WHEN get_json_object(normal_log, '$.old') IN ('Active','Screen Saver','Screen Off')
           AND get_json_object(normal_log, '$.new') IN ('Active Standby','Always Ready','Suspend','Power Off','Reboot') THEN 'TV_OFF'
    END AS ev
  FROM kic_data_ods.tlamp.normal_log_webos25
  WHERE date_ym = '2025-06'
    AND context_name = 'tvpowerd'
    AND message_id = 'NL_POWER_STATE'
    AND log_create_time >= to_date(concat(date_ym, '-01'))
),
ev_only AS (
  SELECT * FROM ev WHERE ev IN ('TV_ON','TV_OFF')
),
paired AS (
  SELECT
    date_ym, ts, mac_addr, X_User_Number, ev,
    LAG(ev) OVER (PARTITION BY mac_addr, X_User_Number ORDER BY ts) AS prev_ev,
    LEAD(IF(ev='TV_OFF', ts, NULL), 1) IGNORE NULLS
      OVER (PARTITION BY mac_addr, X_User_Number ORDER BY ts) AS next_off_time
  FROM ev_only
),
watch_spans AS (
  SELECT
    mac_addr,
    X_User_Number,
    ts AS on_time,
    next_off_time,
    TIMESTAMPDIFF(SECOND, ts, next_off_time) AS duration_sec
  FROM paired
  WHERE ev = 'TV_ON'
    AND (prev_ev IS NULL OR prev_ev <> 'TV_ON')        -- 연속 ON 중 첫 ON만
    AND next_off_time IS NOT NULL                      -- 다음 OFF가 있는 경우만
)
SELECT
  mac_addr,
  X_User_Number,
  SUM(CASE WHEN duration_sec > 0 THEN duration_sec END) AS watch_seconds,
  ROUND(SUM(CASE WHEN duration_sec > 0 THEN duration_sec END)/3600.0, 3) AS watch_hours
FROM watch_spans
GROUP BY mac_addr, X_User_Number
ORDER BY mac_addr, X_User_Number;


In [0]:
%sql
WITH ev AS (
  SELECT
    date_ym,
    CAST(log_create_time AS TIMESTAMP) AS ts,
    mac_addr,
    X_User_Number,
    CASE 
      WHEN get_json_object(normal_log, '$.old') IN ('Init','Active Standby','Always Ready','Suspend')
           AND get_json_object(normal_log, '$.new') IN ('Active') THEN 'TV_ON'
      WHEN get_json_object(normal_log, '$.old') IN ('Active','Screen Saver','Screen Off')
           AND get_json_object(normal_log, '$.new') IN ('Active Standby','Always Ready','Suspend','Power Off','Reboot') THEN 'TV_OFF'
    END AS ev
  FROM kic_data_ods.tlamp.normal_log_webos25
  WHERE date_ym = '2025-06'
    AND context_name = 'tvpowerd'
    AND message_id = 'NL_POWER_STATE'
    AND log_create_time >= to_date(concat(date_ym, '-01'))
),
ev_only AS (
  SELECT * FROM ev WHERE ev IN ('TV_ON','TV_OFF')
),
paired AS (
  SELECT
    date_ym, ts, mac_addr, X_User_Number, ev,
    LAG(ev) OVER (PARTITION BY mac_addr, X_User_Number ORDER BY ts) AS prev_ev,
    LEAD(IF(ev='TV_OFF', ts, NULL), 1) IGNORE NULLS
      OVER (PARTITION BY mac_addr, X_User_Number ORDER BY ts) AS next_off_time
  FROM ev_only
),
watch_spans AS (
  SELECT
    mac_addr,
    X_User_Number,
    ts AS on_time,
    next_off_time,
    TIMESTAMPDIFF(SECOND, ts, next_off_time) AS duration_sec
  FROM paired
  WHERE ev = 'TV_ON'
    AND (prev_ev IS NULL OR prev_ev <> 'TV_ON')        -- 연속 ON 중 첫 ON만
    AND next_off_time IS NOT NULL                      -- 다음 OFF가 있는 경우만
)
SELECT
  date(on_time) AS dt,
  mac_addr,
  X_User_Number,
  SUM(CASE WHEN duration_sec > 0 THEN duration_sec END) AS watch_seconds,
  ROUND(SUM(CASE WHEN duration_sec > 0 THEN duration_sec END)/3600.0, 3) AS watch_hours
FROM watch_spans
GROUP BY date(on_time), mac_addr, X_User_Number
ORDER BY dt, mac_addr, X_User_Number;


In [0]:
%sql
-- 1) 원본 + 이벤트만 정제
WITH events AS (
  SELECT
    date_ym,
    CAST(log_create_time AS TIMESTAMP)         AS ts,
    mac_addr,
    X_User_Number,
    CASE 
      WHEN (get_json_object(normal_log, '$.old') IN ('Init','Active Standby','Always Ready','Suspend')
            AND get_json_object(normal_log, '$.new') IN ('Active')) THEN 'TV_ON'
      WHEN (get_json_object(normal_log, '$.old') IN ('Active','Screen Saver','Screen Off')
            AND get_json_object(normal_log, '$.new') IN ('Active Standby','Always Ready','Suspend','Power Off','Reboot')) THEN 'TV_OFF'
      ELSE NULL
    END AS tv_onoff_event
  FROM kic_data_ods.tlamp.normal_log_webos25
  WHERE date_ym = '2025-06'
    AND context_name = 'tvpowerd'
    AND message_id = 'NL_POWER_STATE'
    AND log_create_time >= to_date(concat(date_ym, '-01'))
),
ev_only AS (
  SELECT *
  FROM events
  WHERE tv_onoff_event IN ('TV_ON','TV_OFF')
),
-- 2) 다음 OFF 시간 채우기(같은 기기/유저 파티션에서 다음으로 등장하는 OFF)
paired AS (
  SELECT
    date_ym,
    ts,
    mac_addr,
    X_User_Number,
    tv_onoff_event,
    -- TV_OFF인 경우에만 ts를 채우고, 그 외는 NULL
    CASE WHEN tv_onoff_event = 'TV_OFF' THEN ts END AS off_time,
    -- NULL을 건너뛰고 다음으로 나타나는 off_time을 가져옴
    LEAD(CASE WHEN tv_onoff_event = 'TV_OFF' THEN ts END, 1) IGNORE NULLS
      OVER (PARTITION BY mac_addr, X_User_Number ORDER BY ts) AS next_off_time
  FROM ev_only
),
-- 3) TV_ON인 순간에 대해 다음 OFF까지 시청 시간 계산
watch_spans AS (
  SELECT
    date_ym,
    mac_addr,
    X_User_Number,
    ts AS on_time,
    next_off_time,
    -- 초 단위 시청 시간
    TIMESTAMPDIFF(SECOND, ts, next_off_time) AS duration_sec
  FROM paired
  WHERE tv_onoff_event = 'TV_ON'
)
select *
from watch_spans
where 1=1
  and mac_addr = '0005c60dad06e469a228353fcf9f08bb4cedaf3aa5475c9c64818fe69e139141'

In [0]:
%sql
  SELECT
    date_ym,
    CAST(log_create_time AS TIMESTAMP)         AS ts,
    mac_addr,
    X_User_Number,
    CASE 
      WHEN (get_json_object(normal_log, '$.old') IN ('Init','Active Standby','Always Ready','Suspend')
            AND get_json_object(normal_log, '$.new') IN ('Active')) THEN 'TV_ON'
      WHEN (get_json_object(normal_log, '$.old') IN ('Active','Screen Saver','Screen Off')
            AND get_json_object(normal_log, '$.new') IN ('Active Standby','Always Ready','Suspend','Power Off','Reboot')) THEN 'TV_OFF'
      ELSE NULL
    END AS tv_onoff_event
  FROM kic_data_ods.tlamp.normal_log_webos25
  WHERE date_ym = '2025-06'
    AND context_name = 'tvpowerd'
    AND message_id = 'NL_POWER_STATE'
    AND log_create_time >= to_date(concat(date_ym, '-01'))
    and mac_addr = '0005c60dad06e469a228353fcf9f08bb4cedaf3aa5475c9c64818fe69e139141'
  order by ts

In [0]:

WITH ev AS (
  SELECT
    date_ym,
    CAST(log_create_time AS TIMESTAMP) AS ts,
    mac_addr,
    X_User_Number,
    CASE 
      WHEN get_json_object(normal_log, '$.old') IN ('Init','Active Standby','Always Ready','Suspend')
           AND get_json_object(normal_log, '$.new') IN ('Active') THEN 'TV_ON'
      WHEN get_json_object(normal_log, '$.old') IN ('Active','Screen Saver','Screen Off')
           AND get_json_object(normal_log, '$.new') IN ('Active Standby','Always Ready','Suspend','Power Off','Reboot') THEN 'TV_OFF'
    END AS ev
  FROM kic_data_ods.tlamp.normal_log_webos25
  WHERE date_ym = '2025-06'
    AND context_name = 'tvpowerd'
    AND message_id = 'NL_POWER_STATE'
    AND log_create_time >= to_date(concat(date_ym, '-01'))
      and mac_addr = '0005c60dad06e469a228353fcf9f08bb4cedaf3aa5475c9c64818fe69e139141'
),
ev_only AS (
  SELECT * FROM ev WHERE ev IN ('TV_ON','TV_OFF')
),
paired AS (
  SELECT
    date_ym, ts, mac_addr, X_User_Number, ev,
    LAG(ev) OVER (PARTITION BY mac_addr, X_User_Number ORDER BY ts) AS prev_ev,
    LEAD(IF(ev='TV_OFF', ts, NULL), 1) IGNORE NULLS
      OVER (PARTITION BY mac_addr, X_User_Number ORDER BY ts) AS next_off_time
  FROM ev_only
),
watch_spans AS (
  SELECT
    mac_addr,
    X_User_Number,
    ts AS on_time,
    next_off_time,
    TIMESTAMPDIFF(SECOND, ts, next_off_time) AS duration_sec
  FROM paired
  WHERE ev = 'TV_ON'
    AND (prev_ev IS NULL OR prev_ev <> 'TV_ON')        -- 연속 ON 중 첫 ON만
    AND next_off_time IS NOT NULL                      -- 다음 OFF가 있는 경우만
)

select *
from ev_only

In [0]:
WITH ev AS (
  SELECT
    date_ym,
    CAST(log_create_time AS TIMESTAMP) AS ts,
    mac_addr,
    X_User_Number,
    CASE 
      WHEN get_json_object(normal_log, '$.old') IN ('Init','Active Standby','Always Ready','Suspend')
           AND get_json_object(normal_log, '$.new') IN ('Active') THEN 'TV_ON'
      WHEN get_json_object(normal_log, '$.old') IN ('Active','Screen Saver','Screen Off')
           AND get_json_object(normal_log, '$.new') IN ('Active Standby','Always Ready','Suspend','Power Off','Reboot') THEN 'TV_OFF'
    END AS ev
  FROM kic_data_ods.tlamp.normal_log_webos25
  WHERE date_ym = '2025-06'
    AND context_name = 'tvpowerd'
    AND message_id = 'NL_POWER_STATE'
    AND log_create_time >= to_date(concat(date_ym, '-01'))
      -- and mac_addr = '0005c60dad06e469a228353fcf9f08bb4cedaf3aa5475c9c64818fe69e139141'
),
ev_only AS (
  SELECT
    date_ym,
    CAST(ts AS TIMESTAMP) AS ts,              -- 이미 ts가 있으면 캐스팅만
    mac_addr,
    X_User_Number,
    ev
  FROM ev 
  WHERE ev IN ('TV_ON','TV_OFF')
),
with_prev AS (
  SELECT
    date_ym, ts, mac_addr, X_User_Number, ev,
    LAG(ev) OVER (PARTITION BY mac_addr, X_User_Number ORDER BY ts) AS prev_ev,
    LAG(IF(ev='TV_ON', ts, NULL), 1) IGNORE NULLS
      OVER (PARTITION BY mac_addr, X_User_Number ORDER BY ts) AS prev_on_time
  FROM ev_only
),
off_spans AS (
  SELECT
    mac_addr,
    X_User_Number,
    prev_on_time AS on_time,
    ts           AS off_time,
    TIMESTAMPDIFF(SECOND, prev_on_time, ts) AS duration_sec
  FROM with_prev
  WHERE ev = 'TV_OFF'
    AND prev_on_time IS NOT NULL           -- 직전 ON이 있는 OFF만
    AND (prev_ev IS NULL OR prev_ev <> 'TV_OFF') -- 연속 OFF 제거
    AND ts > prev_on_time                  -- 음수 방지
)
-- SELECT
--   mac_addr,
--   X_User_Number,
--   SUM(duration_sec) AS watch_seconds,
--   ROUND(SUM(duration_sec)/3600.0, 3) AS watch_hours
-- FROM off_spans
-- GROUP BY mac_addr, X_User_Number
-- ORDER BY mac_addr, X_User_Number;
SELECT
  date(on_time) AS dt,
  mac_addr,
  X_User_Number,
  SUM(CASE WHEN duration_sec > 0 THEN duration_sec END) AS watch_seconds,
  ROUND(SUM(CASE WHEN duration_sec > 0 THEN duration_sec END)/3600.0, 3) AS watch_hours
FROM off_spans
GROUP BY date(on_time), mac_addr, X_User_Number
ORDER BY dt, mac_addr, X_User_Number
;


In [0]:
  SELECT
    date_ym,
    CAST(log_create_time AS TIMESTAMP) AS ts,
    mac_addr,
    X_User_Number,
    CASE 
      WHEN get_json_object(normal_log, '$.old') IN ('Init','Active Standby','Always Ready','Suspend')
           AND get_json_object(normal_log, '$.new') IN ('Active') THEN 'TV_ON'
      WHEN get_json_object(normal_log, '$.old') IN ('Active','Screen Saver','Screen Off')
           AND get_json_object(normal_log, '$.new') IN ('Active Standby','Always Ready','Suspend','Power Off','Reboot') THEN 'TV_OFF'
    END AS ev
  FROM kic_data_ods.tlamp.normal_log_webos25
  WHERE date_ym = '2025-06'
    AND context_name = 'tvpowerd'
    AND message_id = 'NL_POWER_STATE'
    AND log_create_time >= to_date(concat(date_ym, '-01'))
    and mac_addr ='0481637383f5035b5a38d1dd2e50d3d5a1f0db42c368622e6a85f35b1fbb3b19' --='0357615b77d65d34c64bc57f602fe30bc06a49265e445fc7c114bffcde787f2e'

In [0]:
WITH ev AS (
  SELECT
    date_ym,
    CAST(log_create_time AS TIMESTAMP) AS ts,
    -- mac_addr,
    X_User_Number,
    CASE 
      WHEN get_json_object(normal_log, '$.old') IN ('Init','Active Standby','Always Ready','Suspend')
           AND get_json_object(normal_log, '$.new') IN ('Active') THEN 'TV_ON'
      WHEN get_json_object(normal_log, '$.old') IN ('Active','Screen Saver','Screen Off')
           AND get_json_object(normal_log, '$.new') IN ('Active Standby','Always Ready','Suspend','Power Off','Reboot') THEN 'TV_OFF'
    END AS ev
  FROM kic_data_ods.tlamp.normal_log_webos25
  WHERE date_ym BETWEEN '2025-06' and '2025-08'
    AND context_name = 'tvpowerd'
    AND message_id = 'NL_POWER_STATE'
    AND log_create_time >= to_date(concat(date_ym, '-01'))
    AND X_User_Number != ''
    AND X_Device_Country = 'KR'
),
ev_only AS (
  SELECT
    date_ym,
    CAST(ts AS TIMESTAMP) AS ts,              -- 이미 ts가 있으면 캐스팅만
    -- mac_addr,
    X_User_Number,
    ev
  FROM ev 
  WHERE ev IN ('TV_ON','TV_OFF')
),
with_prev AS (
  SELECT
    date_ym, ts, 
    -- mac_addr, 
    X_User_Number, ev,
    LAG(ev) OVER (PARTITION BY X_User_Number ORDER BY ts) AS prev_ev, -- mac_addr,
    LAG(IF(ev='TV_ON', ts, NULL), 1) IGNORE NULLS
      OVER (PARTITION BY  X_User_Number ORDER BY ts) AS prev_on_time -- mac_addr,
  FROM ev_only
),
off_spans AS (
  SELECT
    -- mac_addr,
    X_User_Number,
    prev_on_time AS on_time,
    ts           AS off_time,
    TIMESTAMPDIFF(SECOND, prev_on_time, ts) AS duration_sec
  FROM with_prev
  WHERE ev = 'TV_OFF'
    AND prev_on_time IS NOT NULL           -- 직전 ON이 있는 OFF만
    AND (prev_ev IS NULL OR prev_ev <> 'TV_OFF') -- 연속 OFF 제거
    AND ts > prev_on_time                  -- 음수 방지
), user_num_dimension AS ( 
  select X_User_Number as mbr_no, X_User_Number_hashed as X_User_Number
  from kic_data_private.tlamp.rbt_x_user_number_webos22
  union all
  select X_User_Number as mbr_no, X_User_Number_hashed as X_User_Number
  from kic_data_private.tlamp.rbt_x_user_number_webos23
  union all
  select X_User_Number as mbr_no, X_User_Number_hashed as X_User_Number
  from kic_data_private.tlamp.rbt_x_user_number_webos24
  union all
  select X_User_Number as mbr_no, X_User_Number_hashed as X_User_Number
  from kic_data_private.tlamp.rbt_x_user_number_webos25
  union all
  select X_User_Number as mbr_no, X_User_Number_hashed as X_User_Number
  from kic_data_private.tlamp.rbt_x_user_number_webos60
), distinct_user_num_dimension AS (
  select distinct mbr_no, X_User_Number
  from user_num_dimension
)
SELECT
  date(on_time) AS dt,
  mbr_no,
  SUM(CASE WHEN duration_sec > 0 THEN duration_sec END) AS watch_seconds,
  ROUND(SUM(CASE WHEN duration_sec > 0 THEN duration_sec END)/3600.0, 2) AS watch_hours,
  ROUND(SUM(CASE WHEN duration_sec > 0 THEN duration_sec END)/3600.0/24, 2) AS watch_days
FROM off_spans
LEFT JOIN distinct_user_num_dimension 
  using (X_User_Number)
GROUP BY date(on_time), mbr_no
ORDER BY dt, mbr_no
;


In [0]:
select mac_dimension as (
  select 
    X_User_Number as mbr_no, X_User_Number_hashed as X_User_Number
  from kic_data_private.tlamp.rbt_x_user_number_webos22
  X_User_Number_hashed
  union all
  select *
  from kic_data_private.tlamp.rbt_x_user_number_webos23
  union all
  select *
  from kic_data_private.tlamp.rbt_x_user_number_webos24
  union all
  select *
  from kic_data_private.tlamp.rbt_x_user_number_webos25
  union all
  select *
  from kic_data_private.tlamp.rbt_x_user_number_webos60
)
